In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from keras_tuner.applications import HyperResNet
import keras_tuner
import tensorflow as tf
import matplotlib

import dataset

In [ ]:
 # read dataset

# Normalize pixel values between 0 and 1
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

#to categorical (?)

parameter tuning

In [ ]:
def build_model():
  model = models.Sequential()
  hp = keras_tuner.HyperParameters()

  #add input layer
  model.add(layers.Input(shape=(?,?,?)))

  #add data augmentation layer???

  # Tune the number of layers.
  for i in range(hp.Int("num_layers", min_value = 1, max_value = 64, step = 8)):

      #tune filter size
      hp_filters = hp.Int(f"conv_filter_{i}", min_value=32, max_value=64, step=16)
      #tune filter size. Most used are 3x3 and 5x5
      hp_kernel_size=hp.Choice("conv_kernel", values = [3,5])
      #add convolutional and pooling layer
      model.add(layers.Conv2D(filters = hp_filters, kernel_size = (hp_kernel_size,hp_kernel_size), activation='relu'))
      model.add(layers.MaxPooling2D((2, 2)))
  
  #decide whether to use dropout or not
  if hp.Boolean("dropout"):
      model.add(layers.Dropout(rate=0.25))

  #add classification layer
  model.add(layers.Dense(10, activation="softmax"))

  #tune learning rate
  hp_learning_rate = hp.Float("learning rate", min_value=1e-4, max_value=1e-2, sampling="log") #decide other values?
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss="categorical_crossentropy",
      metrics=["accuracy"],
  )

  return model

 train with optimized parameters

In [ ]:
tuner = keras_tuner.Hyperband(. #decide objective and type of tuner
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='galaxy_dir',
    project_name='galaxy_classification_project'
)

#early stopping callbacks
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5) 
tensorboard = keras.callbacks.TensorBoard(log_dir)

#find best parameters
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

#tune model with optimal parameters
model = tuner.hypermodel.build(best_hps)

Possibile test con entrambi gli approcci ?

In [ ]:
tuner = keras_tuner.RandomSearch(
    HyperResNet(input_shape=(?, ?, ?), classes=10), #used a in-built model for tuning, useful in CV applications.
                                                      #there is no need to define a search space so users can use it directly
    objective='val_accuracy', #decide objective
    max_epochs=10,
    directory='galaxy_dir',  #store search result in this dir
    project_name='galaxy_classification_project' # will be a sub dir of the above directory
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tensorboard = keras.callbacks.TensorBoard(log_dir)
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early, tensorboard])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

In [ ]:
#test models